In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
def read_tsv_file(file_path, headers = True):
    try:
        data = []
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            # Using the CSV reader with the tab delimiter
            reader = csv.reader(file, delimiter='\t')

            if headers == True:
                # Reading headers
                headers = next(reader)
                print("Headers:", headers)

            # Reading data rows
            for row in reader:
                data.append(row)

        # Create a Pandas DataFrame using the headers and data
        if headers == True:
            df = pd.DataFrame(data, columns=headers)
        else:
            df = pd.DataFrame(data) #, columns=headers)
        return df

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
dataset_path = "/home/samuel/NYU/BDS/project/meddra/"
meddra_file_path = dataset_path + 'meddra_all_se.tsv'
meddra_df = read_tsv_file(meddra_file_path, headers = False)

In [4]:
drug_names_file_path = "/home/samuel/NYU/BDS/project/sider/" + 'drug_names.tsv'
drug_names_df = read_tsv_file(drug_names_file_path, headers = False)
drug_names_df = drug_names_df.rename(columns={0: "CID", 1: "drug_name"})

In [5]:
processed_dataset = pd.read_csv("../review_dataset/drugsComTrain_processed.csv")


In [13]:
drug_counts = processed_dataset['drugName'].value_counts()[processed_dataset['drugName'].value_counts() > 100].reset_index().rename(columns={'drugName': 'drug_name', 'count': 'count'})
drug_counts['drug_name_lower'] = drug_counts['drug_name'].str.lower()

merged_df = pd.merge(drug_counts, drug_names_df, left_on='drug_name_lower', right_on="drug_name", how='inner')

merged_df.drop(['drug_name_lower','drug_name_y'], axis=1, inplace=True)
merged_df = merged_df.rename(columns = {"drug_name_x": "drug_name"})

merged_df

,drug_name,count,CID
0,Levonorgestrel,3657,CID100004542
1,Phentermine,1543,CID100004771
2,Sertraline,1360,CID100005203
3,Gabapentin,1047,CID100003446
4,Bupropion,1022,CID100000444
...,...,...,...
148,Celecoxib,109,CID100002662
149,Cetirizine,108,CID100002678
150,Paliperidone,104,CID100115237
151,Melatonin,103,CID100000896


In [14]:
from fuzzywuzzy import fuzz
import pandas as pd

# Assuming drug_counts and drug_names_df are your DataFrames
# and you have already created the 'drug_name_lower' column in drug_counts

# Function to perform approximate matching
def fuzzy_merge(df1, df2, key1, key2, threshold=90):
    """
    Perform fuzzy matching between two DataFrames on specified columns.
    """
    matched_pairs = []
    for idx1, row1 in df1.iterrows():
        print(row1)
        for idx2, row2 in df2.iterrows():
            if fuzz.partial_ratio(row1[key1], row2[key2]) >= threshold:
                matched_pairs.append((idx1, idx2))
    return matched_pairs

# Call the fuzzy_merge function to perform approximate matching
matched_indices = fuzzy_merge(drug_counts, drug_names_df, 'drug_name_lower', 'drug_name', threshold=90)

# Create a DataFrame with the matched indices
matched_df = pd.DataFrame(matched_indices, columns=['idx1', 'idx2'])

# Merge drug_counts and drug_names_df using the matched indices
merged_df = pd.merge(matched_df, drug_counts, left_on='idx1', right_index=True)
merged_df = pd.merge(merged_df, drug_names_df, left_on='idx2', right_index=True)

# Drop unnecessary columns
merged_df.drop(['idx1', 'idx2', 'drug_name_lower', 'drug_name_y'], axis=1, inplace=True)
merged_df.rename(columns={"drug_name_x": "drug_name"}, inplace=True)

print(merged_df)


drug_name          Levonorgestrel
count                        3657
drug_name_lower    levonorgestrel
Name: 0, dtype: object
drug_name          Etonogestrel
count                      3336
drug_name_lower    etonogestrel
Name: 1, dtype: object
drug_name          Ethinyl estradiol / norethindrone
count                                           2850
drug_name_lower    ethinyl estradiol / norethindrone
Name: 2, dtype: object
drug_name          Nexplanon
count                   2156
drug_name_lower    nexplanon
Name: 3, dtype: object
drug_name          Ethinyl estradiol / norgestimate
count                                          2117
drug_name_lower    ethinyl estradiol / norgestimate
Name: 4, dtype: object
drug_name          Ethinyl estradiol / levonorgestrel
count                                            1888
drug_name_lower    ethinyl estradiol / levonorgestrel
Name: 5, dtype: object
drug_name          Phentermine
count                     1543
drug_name_lower    phentermine
Name: 6